# Assignment 1: Data Parsing, Cleansing and Integration
## Task 1 and 2
#### Student Name: Harold Davies
#### Student ID: 3997902

Date: 17/04/2024

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used:
* pandas
* re
* numpy
* xml.etree.ElementTree

## Introduction
The goal of task 1 and 2 is to parse an XML file containing 50,703 job listings, clean the data including imputing missing salary information, and export the resulting data frame to csv, keeping a log of all changes made. The XML file was found to have 5 layers, including the root node and the data was parsed into a pandas data frame containing 11 columns and 50,703 rows. The first data cleaning task completed was formatting the missing values, we needed missing values from the Company, ContractType and ContractTime columns to be replaced with "non-specified". One of the challenges here were identifying the various data values which corresponded to missing values, and the best approach to this seemed to be filtering through an exported csv version of the data in MSExcel to identify irregular values. Missing values from the Salary column needed to be imputed with numerical values, so missing values in this column were all replaced with None so that they would be easily identifiable later in the project. Salary had many irregularities with values expressed in forms such as # To #, #K or #.# per hour, to name just a few. It was required to use a function leveraging regular expressions to extract the numeric values from each format, calculate consistent annual salary figures and impute these figures back into the salary column in place of the irregularly formatted values. The salary column also had a couple of unrealistic outliers of 10,000,000 with the next higest salary being 150,000 so these were replaced with None values to be replaced later with imputed values. Ten of the unique values in the Location column were found to contain typos, spelling errors or inapprpriate use of case, these were logged and fixed. Id's and Dates needed their data types corrected, but first there was one date identified with a month value over 12 which was changed to 12, apart from that there were no issues with these columns with all the Ids being integer values with 8 digits. Finally, the missing Salary values were imputed using the mean of Salary for the relevant category, and 2 duplicate rows were identified and removed. 

## Importing libraries 

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import re
import numpy as np

## Task 1. Parsing Data

### 1.1. Examining and loading data
To begin with, in order to examine the data, I searched through the root, child and sub-child levels of the xml file, printing the label of the level, element tag, attributes and text content. However,  I discovered that this was not displaying the entire picture, and I wasn't sure how many layers the xml file had, so I wrote a recursive function to continue drilling down deeper until the the lowest layer has had its details printed. I found the XML had 5 layers, so I labelled them appropriately in my printed output so I could easily examine the shape of the tree structure and its contents. Looking at the printed XML structure, it was not clear to me whether there would be additional child elements besides Source, or whether there were additional SourceNames besides MyUkJobs, so I also wrote a function to check for child nodes with a tag other than Source or a subchild node with tag SourceName and text value besides MyUkJobs. When I ran this I discovered that Source was the only child of JobAds, however there were many different SourceNames which would need to be included when I transformed the format of the data. 

In [ ]:
#save XML file
tree = ET.parse('input_file.xml')
root = tree.getroot()

In [ ]:
#function to explore and print the structure
def explore_element(element, level=0):
    #labels for different levels
    labels = {0: "Root", 1: "Child", 2: "Subchild", 3: "Sub-subchild", 4: "Sub-sub-subchild"}
    
    #tag and attributes with label
    print(f"{labels[level]} Element: {element.tag}")
    for key, value in element.attrib.items():
        print(f"{labels[level]} Attribute - {key}: {value}")
    
    #text content if any
    if element.text and element.text.strip():
        print(f"{labels[level]} Text Content: {element.text.strip()}")
    
    #recursively explore child elements
    for child in element:
        explore_element(child, level+1)

#structure
explore_element(root)

In [ ]:
def check_structure(element, level=0):
    #if child element is not "Source"
    if level == 1 and element.tag != "Source":
        print("Warning: Other child elements present besides 'Source'.")

    #if SourceName is MyUkJobs
    if level == 2 and element.tag == "SourceName" and element.text.strip() != "MyUkJobs":
        print("Warning: SourceName is not 'MyUkJobs'. It is: " + element.text)

    #recursively explore child elements
    for child in element:
        check_structure(child, level+1)

check_structure(root)

### 1.2 Parsing data into the required format

Now that I know the shape and contents of the XML, I can extract the data into a list of libraries which will correspond to rows in my new data frame. I am extracting the values corresponding to the sub-subchild level (Id, Title, Location, Company, ContractType, ContractTime, Category and Salary) and sub-sub-subchild level (OpenDate and CloseDate) for every Advertisement subchild, as well as obtaining the SourceName from the parent Source element. When I originally wrote this code, I did not account for potential missing values, and I got errors corresponding to None values in the Company, ContractType, ContractTime and Salary columns, therefore I needed to write if statements for these elements, and will explore the nature of these missing values later. 

In [ ]:
#initiate list for rows
data = []
#for each advertisement, store the appropriate values in a library and append it to the data list
for advertisement in root.findall('.//Advertisement'):
    #initiate library for cell values
    ad_data = {}
    
    #extract data for each column
    ad_data['Id'] = advertisement.find('.//Id').text
    ad_data['Title'] = advertisement.find('.//Title').text
    ad_data['Location'] = advertisement.find('.//Location').text

    company_element = advertisement.find('.//Company')
    if company_element is not None:
        ad_data['Company'] = company_element.text
    else:
        ad_data['Company'] = None

    contract_type_element = advertisement.find('.//ContractType')
    if contract_type_element is not None:
        ad_data['ContractType'] = contract_type_element.text
    else:
        ad_data['ContractType'] = None
    
    contract_time_element = advertisement.find('.//ContractTime')
    if contract_time_element is not None:
        ad_data['ContractTime'] = contract_time_element.text
    else:
        ad_data['ContractTime'] = None

    ad_data['Category'] = advertisement.find('.//Category').text
    
    salary_element = advertisement.find('.//Salary')
    if salary_element is not None:
        ad_data['Salary'] = salary_element.text
    else:
        ad_data['Salary'] = None

    date_element = advertisement.find('.//Date')
    ad_data['OpenDate'] = date_element.find('.//OpenDate').text
    ad_data['CloseDate'] = date_element.find('.//CloseDate').text
    
    #get SourceName from the parent Source element
    source_name = root.find('.//Source/SourceName')
    if source_name is not None:
        ad_data['SourceName'] = source_name.text
    else:
        ad_data['SourceName'] = None
    
    #finally, append the extracted data to the list
    data.append(ad_data)


In [ ]:
#convert to pandas data frame
df = pd.DataFrame(data, columns=['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'])

## Task 2. Auditing and cleansing the loaded data

We will start by exporting the data frame to csv to assist in exploring the data values, and then create a log to track changes made

In [ ]:
df.to_csv('raw_data.csv', index=False)

In [ ]:
df.shape

In [ ]:
df.head()

### Change log

In [ ]:
#create change log
itemlist = ['indexOfdf','ColumnName', 'Orignal', 'Modified', 'ErrorType','Fixing']
erlist = pd.DataFrame(columns=itemlist)
erlist

In [ ]:
#function for updating change log
def updateErlist(indexOfdf, ColumnName, Orignal, Modified, ErrorType, Fixing):
    #create list from input data
    errItem=[indexOfdf,ColumnName, Orignal, Modified, ErrorType,Fixing]
    #add list to the end of the error log
    erlist.loc[len(erlist)]=errItem

In [ ]:
#generate indices to referencxe in the error log
indices = df.index 
indices

### Missing values

Four columns contain missing values. There are also other missing values represented by other irregular values. 

In [ ]:
df.isna().sum()

#### Company

Missing values in the Company column include cells containing "", -, N/A and " ", as well as 2 numerical values 21 and 591. Turns out there is a hospitality and catering company in London called "21", but there is no company in London called "591".

In [ ]:
#strip leading and trailing spaces 
df['Company'] = df['Company'].str.strip()

In [ ]:
#mask locations of missing values
condition = df.Company.isna()
#get list of indices of missing values
applied_indicies = indices[condition]
rows_added = 0
#for each missing value, update the change log
for ind in applied_indicies:
    updateErlist(ind, 'Company', None, 'non-specified', 'missing value', "replacing all None with 'non-specified'")
    rows_added += 1
print(str(rows_added) + " rows added to change log")
#change missing values to 'non-specified'
df['Company'][condition] = 'non-specified'

In [ ]:
(df['Company'] == '-').sum()

In [ ]:
#replace "-" values
condition = df.Company == '-'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'Company', '-', 'non-specified', 'missing value', "replacing all '-' with 'non-specified'")
df['Company'][condition] = 'non-specified'

In [ ]:
(df['Company'] == '').sum()

In [ ]:
#replace "" values
condition = df.Company == ''
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'Company', '', 'non-specified', 'missing value', "replacing all '' with 'non-specified'")
df['Company'][condition] = 'non-specified'

In [ ]:
(df['Company'] == "N/A").sum()

In [ ]:
#replace "N/A" values
condition = df.Company == 'N/A'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'Company', 'N/A', 'non-specified', 'missing value', "replacing all 'N/A' with 'non-specified'")
df['Company'][condition] = 'non-specified'

In [ ]:
(df['Company'] == ".").sum()

In [ ]:
#lookup indices of other irregular values
condition = ((df.Company == '.') | (df.Company == '591'))
applied_indicies = indices[condition]
df[condition]['Company']

In [ ]:
#update change log and fix the values
updateErlist(25097, 'Company', '591', 'non-specified', 'typo', "There is no hospitality and catering company in London called '591'")
df.at[25097, 'Company'] = 'non-specified'
updateErlist(42067, 'Company', '.', 'non-specified', 'missing value', "replacing missing value with 'non-specified'")
df.at[42067, 'Company'] = 'non-specified'

#### ContractType

Missing values in the Company column also include cells containing None, -, N/A and " ", we will replace them all with 'non-specified'.

In [ ]:
df['ContractType'].unique()

In [ ]:
#strip leading and trailing spaces
df['ContractType'] = df['ContractType'].str.strip()

In [ ]:
(df['ContractType'] == '').sum()

In [ ]:
#replace "" values
condition = df.ContractType == ''
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'ContractType', '', 'non-specified', 'missing value', "replacing all '' with 'non-specified'")
df['ContractType'][condition] = 'non-specified'

In [ ]:
#replace None values
condition = df.ContractType.isna()
applied_indicies = indices[condition]
rows_added = 0
for ind in applied_indicies:
    updateErlist(ind, 'ContractType', None, 'non-specified', 'missing value', "replacing all None with 'non-specified'")
    rows_added += 1
print(str(rows_added) + " rows added to change log")
df['ContractType'][condition] = 'non-specified'

In [ ]:
(df['ContractType'] == '-').sum()

In [ ]:
#replace "-" values
condition = df.ContractType == '-'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'ContractType', '-', 'non-specified', 'missing value', "replacing all '-' with 'non-specified'")
df['ContractType'][condition] = 'non-specified'

In [ ]:
(df['ContractType'] == "N/A").sum()

In [ ]:
#replace "N/A" values
condition = df.ContractType == 'N/A'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'ContractType', 'N/A', 'non-specified', 'missing value', "replacing all 'N/A' with 'non-specified'")
df['ContractType'][condition] = 'non-specified'

#### ContractTime

Missing values in the ContractTime column also include cells containing None, -, N/A and " ", again, we will replace them with 'non-specified'.

In [ ]:
df['ContractTime'].unique()

In [ ]:
#strip leading and trailing spaces
df['ContractTime'] = df['ContractTime'].str.strip()

In [ ]:
#replace None values
condition = df.ContractTime.isna()
applied_indicies = indices[condition]
rows_added = 0
for ind in applied_indicies:
    updateErlist(ind, 'ContractTime', None, 'non-specified', 'missing value', "replacing all None with 'non-specified'")
    rows_added += 1
print(str(rows_added) + " rows added to change log")
df['ContractTime'][condition] = 'non-specified'

In [ ]:
(df['ContractTime'] == '-').sum()

In [ ]:
#replace "-" values
condition = df.ContractTime == '-'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'ContractTime', '-', 'non-specified', 'missing value', "replacing all '-' with 'non-specified'")
df['ContractTime'][condition] = 'non-specified'

In [ ]:
(df['ContractTime'] == "N/A").sum()

In [ ]:
#replace "N/A" values
condition = df.ContractTime == 'N/A'
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'ContractTime', 'N/A', 'non-specified', 'missing value', "replacing all 'N/A' with 'non-specified'")
df['ContractTime'][condition] = 'non-specified'

#### Salary

Missing values in the Salary column include " ", "-", None and N/A

In [ ]:
(df['Salary'] == " ").sum()

In [ ]:
(df['Salary'] == "-").sum()

In [ ]:
(df['Salary'].isna()).sum()

In [ ]:
(df['Salary'] == "N/A").sum()

In [ ]:
(df['Salary'] == "").sum()

In [ ]:
#replace " ", "-" and "N/A" values
condition = (df.Salary == ' ') | (df.Salary == '-') | (df.Salary == 'N/A')
applied_indicies = indices[condition]
for ind in applied_indicies:
    updateErlist(ind, 'Salary', 'N/A or - or " "', None, 'missing value', "replacing all 'N/A' with 'non-specified'")
df['Salary'][condition] = None

### Irregularities

#### Salary

*********I'm up to here proof reading, commenting and report writing - need to address Salary Outliers


The salary column contains irregularities in the form of different variations of text and numbers to represent the job salary. As seen below, variations include # per Annum, # To #, # - #, #/Year, #K, #.# per hour and #.# p/h. I will calculate averages for # To #, and change all values to a floating point number representing the annual salary for the job. 

In [ ]:
#find where there are non-None and non-numeric values in the 'Salary' column
alphabetic_mask = df['Salary'].apply(lambda x: isinstance(x, str) and any(c.isalpha() or c == '-' for c in x))

#get the relevant rows
values_with_alphabetic = df[alphabetic_mask]

#remove leading numbers to reduce the # of unique values to a manageable level
values_with_alphabetic['Salary'] = values_with_alphabetic['Salary'].str.replace(r'^\d+', '', regex=True)

#show unique non-None and non-numeric values
values_with_alphabetic['Salary'].unique()

In [ ]:
#define function to deal with all cases
def normalised(sal_string):
    #num To num
    result = re.fullmatch(r'(\d+) To (\d+)', sal_string)
    if result:
        salary = (float(result.group(1))+float(result.group(2)))/2
        #print("found # To #: " + str(result[0]) + " converting to: " + str(salary))
        return salary
    #num - num
    result = re.fullmatch(r'(\d+) - (\d+)', sal_string)
    if result:
        salary = (float(result.group(1))+float(result.group(2)))/2
        #print("found # - #: " + str(result[0]) + " converting to: " + str(salary))
        return (float(result.group(1))+float(result.group(2)))/2
    #num( per Annum /Year)
    result = re.fullmatch(r'(\d+)(( per Annum)|(/Year))', sal_string)
    if result:
        #print("found # per Annum or #/Year: " + str(result[0]) + " converting to: " + str(result.group(1)))
        return float(result.group(1))
    #numK
    result = re.fullmatch(r'(\d+)K', sal_string)
    if result:
        #print("found #K: " + str(result[0]) + " converting to: " + str(float(result.group(1))*1000))
        return float(result.group(1))*1000
    #num(.# per hour .# p/h)
    result = re.fullmatch(r'(\d+\.?\d+)( per hour| p/h)', sal_string)
    if result:
        #print("found #.# per hour or #.# p/h: " + str(result[0]) + " converting to: " + str(float(result.group(1))*37.5*52))
        return float(result.group(1))*37.5*52
    #return 

In [ ]:
#define a new column using the function and rounding to 2 decimal places
df['Salary_fixed'] = round(df['Salary'].astype(str).apply(lambda x: normalised(x)), 2)

In [ ]:
#fixed salary values
df.describe()

In [ ]:
#create mask of fixed salary values
condition = pd.notna(df['Salary_fixed'])
#get list of indices where salaries need to be fixed
applied_indices = indices[condition]
#initiate list for updates
update_list = []
#for each index, update the change log and create a tuple to update df
for ind in applied_indices:
    original_salary = df['Salary'].iloc[ind]
    modified_salary = df['Salary_fixed'].iloc[ind]
    updateErlist(ind, 'Salary', original_salary, modified_salary, 'inconsistent formatting', "replacing inconsistently formatted salary with floating point number")
    update_tuple = (ind, 'Salary', modified_salary)
    update_list.append(update_tuple)

#update df
for ind, col, new_value in update_list:
    df.at[ind, col] = new_value

## Outliers

The salary column contains a couple of values equal to 10,000,000 which are quite clearly incorrect with the next highest value being 150,000. I will change these values to None so that they will be imputed with estimated values later in the project. 

In [ ]:
#print salary values over 100k
for value in df['Salary']:
    try:
        value = float(value)
        if value > 100000:
            print(value)
    except:
        pass

In [ ]:
#get indices of outliers
df[df['Salary'] == '10000000']

In [ ]:
#update change log and change values to None
updateErlist(40787, 'Salary', '10000000', None, 'outlier', "replacing extreme/unrealistic outlier with None value to be imputed later")
updateErlist(49589, 'Salary', '10000000', None, 'outlier', "replacing extreme/unrealistic outlier with None value to be imputed later")
df.at[40787, 'Salary'] = None
df.at[49589, 'Salary'] = None

### Typos and Spelling Mistakes

#### Location

In [ ]:
#the tail of the valuecounts shows the infrequently entered typos and mistakes
df['Location'].value_counts().tail(12)

In [ ]:
#get the indices of the errors
condition = ((df.Location == 'Manchaster') | (df.Location == 'BRISTOL') | (df.Location == 'london') | (df.Location == 'ABERDEEN') | (df.Location == 'Livepool') 
             | (df.Location == 'Oxfords') | (df.Location == 'Leads') | (df.Location == 'birmingham') | (df.Location == 'Cembridge') | (df.Location == 'HAMpshire'))
applied_indicies = indices[condition]
df[condition]['Location']

In [ ]:
#for each error, update the log and fix the value
updateErlist(544, 'Location', 'ABERDEEN', 'Aberdeen', 'typo', "replacing with correct case letters")
df.at[544, 'Location'] = 'Aberdeen'
updateErlist(3132, 'Location', 'BRISTOL', 'Bristol', 'typo', "replacing with correct case letters")
df.at[3132, 'Location'] = 'Bristol'
updateErlist(21121, 'Location', 'london', 'London', 'typo', "replacing with correct case letters")
df.at[21121, 'Location'] = 'London'
updateErlist(36396, 'Location', 'Manchaster', 'Manchester', 'typo', "fixing typo")
df.at[36396, 'Location'] = 'Manchester'
updateErlist(47528, 'Location', 'Livepool', 'Liverpool', 'typo', "fixing typo")
df.at[47528, 'Location'] = 'Liverpool'
updateErlist(2664, 'Location', 'birmingham', 'Birmingham', 'typo', "replacing with correct case letters")
df.at[2664, 'Location'] = 'Birmingham'
updateErlist(4048, 'Location', 'Leads', 'Leeds', 'typo', "replacing with correct case letters")
df.at[4048, 'Location'] = 'Leeds'
updateErlist(7604, 'Location', 'Cembridge', 'Cambridge', 'typo', "replacing with correct case letters")
df.at[7604, 'Location'] = 'Cambridge'
updateErlist(19470, 'Location', 'HAMpshire', 'Hampshire', 'typo', "replacing with correct case letters")
df.at[19470, 'Location'] = 'Hampshire'
updateErlist(21941, 'Location', 'HAMpshire', 'Hampshire', 'typo', "replacing with correct case letters")
df.at[21941, 'Location'] = 'Hampshire'
updateErlist(41739, 'Location', 'Cembridge', 'Cambridge', 'typo', "replacing with correct case letters")
df.at[41739, 'Location'] = 'Cambridge'
updateErlist(42678, 'Location', 'Oxfords', 'Oxford', 'typo', "replacing with correct case letters")
df.at[42678, 'Location'] = 'Oxford'

In [ ]:
#check the tail to see the edited values now fall into bigger value counts
df['Location'].value_counts().tail(5)

#### Category

In [ ]:
#Check category value counts - all good
df['Category'].value_counts()

### Data Types

#### Id

In [ ]:
#change Ids to int type
df['Id'] = df['Id'].astype(int)

In [ ]:
#check max and min to ensure all Ids are 8 digits
print("Id min = " + str(df['Id'].min()) + " and Id max = " + str(df['Id'].max()))

#### Dates

Upon attempting to use the to_datetime pandas function to fix the date formats, I discovered there was at least one date not conforming to the observed original format. I use a function to detect any such case and then fix this before going ahead and transforming the format. 

In [ ]:
#define finction to find dates with month > 12
def find_inconsistent_dates(date_string):
    #months > 12
    result = re.fullmatch(r'\d{4}(1[3-9]|[23]\d).*', date_string)
    if result:
        print("found inconsistent date format: " + date_string)
        return date_string

In [ ]:
#execute function on all OpenDate values
for date in df['OpenDate']:
    find_inconsistent_dates(date)

In [ ]:
#find index of troublesome value
df[df['OpenDate'] == '20121612T150000']

In [ ]:
#update change log and fix value
updateErlist(2615, 'OpenDate', '20121612T150000', '20121212T150000', 'violation of integrity constraint', "changing month to 12 instead of 16 (impossible value for month)")
df.at[2615, 'OpenDate'] = '20121212T150000'

In [ ]:
#check CloseDate values
for date in df['CloseDate']:
    find_inconsistent_dates(date)

In [ ]:
#tranform formats
df['OpenDate'] = pd.to_datetime(df['OpenDate'], format='%Y%m%dT%H%M%S')
df['CloseDate'] = pd.to_datetime(df['OpenDate'], format='%Y%m%dT%H%M%S')

In [ ]:
#check data types - Salary should be the final column remaining to be fixed
df.dtypes

#### Salary

In [ ]:
#get rows where Salary is not None
df_salary = df[df['Salary'] != None]

#convert the salary column to floating point numbers
df_salary['Salary'] = df_salary['Salary'].astype(float)

#calculate and store the mean salary value for each Category
category_salary_mean = df_salary.groupby('Category')['Salary'].mean(numeric_only=True)

#merge values onto df
df = df.merge(category_salary_mean, left_on='Category', right_index=True, suffixes=('', '_mean_for_category'))

In [ ]:
#iterate through rows where the salary value is missing, updating the change log and imputing the relevant value
for index, row in df[df['Salary'].isna()].iterrows():
    original_value = None
    modified_value = row['Salary_mean_for_category']
    error_type = 'missing value'
    fixing = "imputing missing value with mean salary for category"
    updateErlist(index, 'Salary', original_value, modified_value, error_type, fixing)
    #impute the 'Salary' column with the mean salary for the corresponding category
    df.at[index, 'Salary'] = modified_value

In [ ]:
#drop ancillary columns which are no longer needed
df.drop(['Salary_fixed', 'Salary_mean_for_category'], axis=1, inplace=True)

In [ ]:
#check for duplicates
df[df.duplicated(subset=df.columns.difference(['Id']), keep=False)]

In [ ]:
#update change log
updateErlist(43706, 'All', df.iloc[43706], 'deleted', 'duplicate row', 'deleted duplicate row')
updateErlist(47601, 'All', df.iloc[47601], 'deleted', 'duplicate row', 'deleted duplicate row')
#delete duplicate rows
df = df.drop(index=[47601, 43706])

......

## Saving data

The data has been parsed and cleaned, with missing values dealt with as per the goal of the project, we will export them to csv for later use. 

In [ ]:
#export cleaned csv
df.to_csv('clean_data.csv', index=False)
#export change log
erlist.to_csv('change_log.csv', index=False)

## Summary

After the data was parsed, many issues needed to be addressed in order to thoroughly clean the data. Four columns needed missing values identified and reformatted, salary had a couple outliers to fix, location had numerous typos, spelling errors and inconsistent use of case, opendate violated an integrity constraint, dates and Ids needed data types adjusted, Salary required calculation and imputation of values to replace missing values and finally there were 2 duplicate rows. The data has now been well cleaned and a change log and the cleaned dataset have been exported. 